In [1]:
import time
import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.utils import uri_helper

In [2]:
URI = 'radio://0/60/2M/E7E7E7E706'  # Change to your drone's URI
target_z = 1.5  # Target height in meters
zKp, zKi, zKd = 2.02, 0.025, 0.7  # PID parameters for Z-axis
xKp, xKi, xKd = 2.0, 0.5, 0.0  # PID parameters for X-axis
yKp, yKi, yKd = 2.0, 0.5, 0.0  # PID parameters for Y-axis

filename = f'C:/Users/omerr/drone_win/data_pid/pid_xy_{xKp}-{xKi}-{xKd}_{yKp}-{yKi}-{yKd}.csv'
figurename = f'pid_xy_{xKp}-{xKi}-{xKd}_{yKp}-{yKi}-{yKd}.jpg'
gifname = f'pid_xy_{xKp}-{xKi}-{xKd}_{yKp}-{yKi}-{yKd}.gif'

sequence = [(2.0,2.0,target_z,0.0),
            (2.0,2.0,target_z-0.5,0.0),
            (2.0,2.0,target_z-1.0,0.0),
            (2.0,2.0,target_z-1.5,0.0)]  # Sequence setpoint for the drone

def activate_mellinger_controller(scf):
    scf.cf.param.set_value('stabilizer.controller', '2')

In [3]:
def take_off(scf):
    commander= scf.cf.high_level_commander

    commander.takeoff(target_z, 3.0)
    time.sleep(5.0)

def land(scf):
    commander = scf.cf.high_level_commander

    commander.land(0.1, 5.0)
    time.sleep(3)

    commander.stop()

def reset_estimator(scf):
    cf = scf.cf
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(2.0)
    cf.param.set_value('kalman.resetEstimation', '0')
    time.sleep(2.0)


def position_callback(timestamp, data, log_conf):
    z = data['kalman.stateZ']
    x = data['kalman.stateX']
    y = data['kalman.stateY']

    with open(filename, 'a') as f:
        f.write('{},{},{},{}\n'.format(timestamp, x, y, z))


def run_sequence_setpoint(scf, sequence):
    cf = scf.cf
    for position in sequence:
        print('Setting position {}'.format(position))
        for i in range(15):
            cf.commander.send_position_setpoint(position[0],
                                                position[1],
                                                position[2],
                                                position[3])
            time.sleep(0.1)


def start_position_printing(scf):
    log_conf = LogConfig(name='Position', period_in_ms=100)
    log_conf.add_variable('kalman.stateX', 'float')
    log_conf.add_variable('kalman.stateY', 'float')
    log_conf.add_variable('kalman.stateZ', 'float')
    
    scf.cf.log.add_config(log_conf)
    log_conf.data_received_cb.add_callback(position_callback)
    log_conf.start() 
    return log_conf  # Return the log configuration for later use

In [4]:
import os

cflib.crtp.init_drivers()
with SyncCrazyflie(URI, cf=Crazyflie(rw_cache='./cache')) as scf:
    # Example: Set pitch PID value
    hover_time = 20
    commander = scf.cf.high_level_commander
    target_x, target_y = 2.0, 2.0

    ################################################
    scf.cf.param.set_value('posCtlPid.zKp', zKp)
    scf.cf.param.set_value('posCtlPid.zKi', zKi)
    scf.cf.param.set_value('posCtlPid.zKd', zKd)

    scf.cf.param.set_value('posCtlPid.xKp', xKp)
    scf.cf.param.set_value('posCtlPid.xKi', xKi)
    scf.cf.param.set_value('posCtlPid.xKd', xKd)

    scf.cf.param.set_value('posCtlPid.yKp', yKp)
    scf.cf.param.set_value('posCtlPid.yKi', yKi)
    scf.cf.param.set_value('posCtlPid.yKd', yKd)
    ################################################


    # Repeat for roll and yaw as needed
    print("PID parameters updated.")
    print("X: Kp={}, Ki={}, Kd={}".format(xKp, xKi, xKd))
    print("Y: Kp={}, Ki={}, Kd={}".format(yKp, yKi, yKd))
    print("Z: Kp={}, Ki={}, Kd={}".format(zKp, zKi, zKd))




    if os.path.exists(filename):
        os.remove(filename)
    time.sleep(1)

    # activate_mellinger_controller(scf)

    print("Resetting estimators...")
    reset_estimator(scf)
    scf.cf.platform.send_arming_request(True)
    time.sleep(1.0)
    print("Taking off...")
    take_off(scf)
    time.sleep(2.5)
    print("Going to position (2.0, 2.0, target_z)...")
    commander.go_to(target_x, target_y, target_z, 0, 3.0, relative=False)
    time.sleep(2.0)  # Wait for the drone to reach the position
    log_conf = start_position_printing(scf)
    print(f'Hovering for {hover_time} seconds...')
    time.sleep(hover_time)
    log_conf.stop()
    print("Landing...")
    run_sequence_setpoint(scf, sequence)
    time.sleep(1.0)  # Wait for the landing to complete
    # land(scf)

PID parameters updated.
X: Kp=2.0, Ki=0.5, Kd=0.0
Y: Kp=2.0, Ki=0.5, Kd=0.0
Z: Kp=2.02, Ki=0.025, Kd=0.7
Resetting estimators...
Taking off...
Going to position (2.0, 2.0, target_z)...
Hovering for 20 seconds...


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

print("Plotting and saving data...")
df = pd.read_csv(filename, header=None, names=['timestamp','x', 'y', 'z'])
df['time_s'] = (df['timestamp'] - df['timestamp'].iloc[0]) / 1000.0  # adjust divisor if needed

fig, axes = plt.subplots(1, 3, figsize=(20, 7), sharex=True)

coords = ['x', 'y', 'z']
labels = ['X Position (m)', 'Y Position (m)', 'Z Position (m)']
titles = ['Drone X Over Time', 'Drone Y Over Time', 'Drone Z Over Time']

for i, (coord, label, title) in enumerate(zip(coords, labels, titles)):
    axes[i].plot(df['time_s'], df[coord], label=f'{coord.upper()} Position')
    mean_val = df[coord].mean()
    var_val = df[coord].var()
    axes[i].set_xlabel('Time (s)')
    axes[i].set_ylabel(label)
    axes[i].set_title(title)
    axes[i].grid()
    axes[i].legend()
    # Annotate mean and variance
    axes[i].text(
        0.5, 0.95,
        f'Mean: {mean_val:.3f} m\nVar: {var_val:.4f}',
        transform=axes[i].transAxes,
        fontsize=12,
        color='darkred',
        ha='center',
        va='top',
        bbox=dict(facecolor='white', alpha=0.7, edgecolor='none')
    )
axes[2].axhline(y=target_z, color='red', linestyle='--', linewidth=2, label='Setpoint = 1.5m')
axes[0].axhline(y=target_x, color='red', linestyle='--', linewidth=2, label=f'Setpoint = {target_x} m')
axes[1].axhline(y=target_y, color='red', linestyle='--', linewidth=2, label=f'Setpoint = {target_y} m')
plt.tight_layout()
plt.savefig(f'C:/Users/omerr/drone_win/data_pid/{figurename}', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
from matplotlib.animation import FuncAnimation

# Extract x and y columns
x = df.iloc[:,1].values
y = df.iloc[:,2].values

# Create figure and axis
fig, ax = plt.subplots(figsize=(6,6))
ax.set_xlim(min(x)-0.5, max(x)+0.5)
ax.set_ylim(min(y)-0.5, max(y)+0.5)
ax.set_xlabel('X position (m)')
ax.set_ylabel('Y position (m)')
ax.set_title('Drone X-Y Trajectory Animation')
ax.grid(True)

# Create line and point objects
line, = ax.plot([], [], lw=2, color='blue')
point, = ax.plot([], [], 'ro')  # red dot for current position

# Initialization function
def init():
    line.set_data([], [])
    point.set_data([], [])
    return line, point

# Update function
def update(frame):
    if frame == 0:
        # Clear at the start
        line.set_data([], [])
        point.set_data([], [])
        return line, point

    line.set_data(x[:frame], y[:frame])
    point.set_data([x[frame-1]], [y[frame-1]])  # <--- Wrap in lists
    return line, point



# Create the animation
anim = FuncAnimation(
    fig,
    update,
    frames=len(x),
    init_func=init,
    blit=True,
    interval=100  # Adjust playback speed here
)

# Save as GIF
anim.save(f'C:/Users/omerr/drone_win/data_pid/{gifname}', writer='pillow', fps=30)

print(f'Animation saved as {gifname}')